In [ ]:
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from datasets import Dataset
warnings.filterwarnings('ignore')

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.metrics import accuracy_score, classification_report
from sklearn.svm import LinearSVC
from sklearn.utils import resample
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    EarlyStoppingCallback,
)
import torch
from datasets import Dataset



In [ ]:
# @title
#Please : i renamed the dfs to add clarity
obama_df = pd.read_excel('/content/training-Obama-Romney-tweets.xlsx', sheet_name='Obama')
romney_df = pd.read_excel('/content/training-Obama-Romney-tweets.xlsx', sheet_name='Romney')


In [ ]:
# @title
def cleaning_and_processing(df):
  #Drof the first completely empty column
  df = df.drop(df.columns[0], axis = 1)
  #rename columns
  df = df.rename(columns={
      'date' : 'date',
      'time' : 'time',
      'Anootated tweet': 'tweet',
      'Unnamed: 4': 'label',
      'Unnamed: 5': 'your_label'
  })
  #remove tags from tweet column -- Please : maybe remove it, we're already doing that in the main tweet cleaning fx?
  df['tweet'] = df['tweet'].str.replace(r'<[^>]+>', '', regex=True)
  #drop second empty/useless row
  df = df.drop(index=1).reset_index(drop=True)
  #Drop your label for now, its useless
  df = df.drop(columns=['your_label'])
  #convert date and time
  '''
  these are raising a lot of errors ad we don't even need it so cmt out
  df['date'] = pd.to_datetime(df['date'], errors='coerce').dt.date
  df['time'] = pd.to_datetime(df['time'], errors='coerce').dt.time
  '''
  df['label'] = df['label'].replace(2, 0)
  df = df[df['label'].isin([-1, 0, 1])].reset_index(drop=True)
  df["clean_tweet"] = df["tweet"].apply(clean_tweet)
  df = df.drop(columns=['tweet'])
  df = df.rename(columns={'clean_tweet': 'tweet'})

  return df

In [ ]:
# will be changed with respect to test data format
def cleaning_and_processing_testdata(df):
  #Drof the first completely empty column
  df = df.drop(df.columns[0], axis = 1)
  #rename columns
  df = df.rename(columns={
      'date' : 'date',
      'time' : 'time',
      'Anootated tweet': 'tweet',
      'Unnamed: 4': 'label',
      'Unnamed: 5': 'your_label'
  })
  #remove tags from tweet column -- Please : maybe remove it, we're already doing that in the main tweet cleaning fx?
  df['tweet'] = df['tweet'].str.replace(r'<[^>]+>', '', regex=True)
  #drop second empty/useless row
  df = df.drop(index=1).reset_index(drop=True)
  #Drop your label for now, its useless
  df = df.drop(columns=['your_label'])
  #convert date and time
  '''
  these are raising a lot of errors ad we don't even need it so cmt out
  df['date'] = pd.to_datetime(df['date'], errors='coerce').dt.date
  df['time'] = pd.to_datetime(df['time'], errors='coerce').dt.time
  '''

  df = df[df['label'].isin([-1, 0, 1])].reset_index(drop=True)
  df["clean_tweet"] = df["tweet"].apply(clean_tweet)
  df = df.drop(columns=['tweet'])
  df = df.rename(columns={'clean_tweet': 'tweet'})
  return df

In [ ]:
# @title
def clean_tweet(text):
    if pd.isna(text):
        #return text #Please : this might return null or nan so i changed it
        return ''
    #convert everything to string
    text = str(text)
    #lowercase everything
    text = text.lower()
    # remove @handles even if stuck to other text
    text = re.sub(r'@\w+', '', text) #Please : added \w+ to remove full mention, not just @
    # remove http, https, or www links even if glued to other text
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    # remove HTML-like tags (<a>, </e>, etc.)
    text = re.sub(r'<.*?>', '', text)
    # remove hashtags symbol but keep the word (optional)
    text = re.sub(r'#', '', text)
    # remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()
    #remove punctuation except letters numbers spaces
    text = re.sub(r'[^a-z0-9\s]', ' ', text)

    return text

In [ ]:
# @title
#apply the cleaning to the data
obama_df = cleaning_and_processing(obama_df)
romney_df = cleaning_and_processing(romney_df)